In [ ]:
# %env OGDF_BUILD_DIR=~/ogdf/build-debug
# uncomment if you didn't set this globally
from ogdf_python import ogdf, cppinclude

cppinclude("ogdf/basic/graph_generators/randomized.h")
cppinclude("ogdf/layered/SugiyamaLayout.h")

G = ogdf.Graph()
ogdf.setSeed(1)
ogdf.randomPlanarTriconnectedGraph(G, 20, 40)
GA = ogdf.GraphAttributes(G, ogdf.GraphAttributes.all)
GA.directed = False

SL = ogdf.SugiyamaLayout()
SL.call(GA)

FIRST = 0
LAST = -1

In [ ]:
def search(G, node1, GA=None, mode=FIRST):
    todo = [node1]
    order = ogdf.NodeArray[int](G, -1)
    count = 0
    while len(todo) > 0:
        cur = todo.pop(mode)
        if order[cur] >= 0:
            continue
            
        order[cur] = count
        if GA:
            GA.label[cur] = str(count)
        count += 1
        
        for adj in cur.adjEntries:
            if order[adj.twinNode()] < 0:
                todo.append(adj.twinNode())
        
        yield todo
    return count

In [ ]:
def dfs(G):
    discovery = ogdf.NodeArray[int](G, -1)
    finish = ogdf.NodeArray[int](G, -1)
    predecessor = ogdf.NodeArray[ogdf.node](G, None)
    
    time = 0
    
    def dfs_visit(u):
        nonlocal time

        time += 1
        discovery[u] = time
        yield u, discovery[u], finish[u]
        
        for adj in u.adjEntries:
            v = adj.twinNode()
            if discovery[node] < 0:
                predecessor[v] = u
                yield from dfs_visit(v)

        time += 1
        finish[u] = time
        yield u, discovery[u], finish[u]
    
    for node in G.nodes:
        if discovery[node] < 0:
            yield from dfs_visit(node)

In [ ]:
it = search(G, G.nodes[0], GA, LAST)
GA

In [ ]:
try:
    print([n.index() for n in next(it)])
except StopIteration as e:
    print("done", e.args)
GA